In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Set constants
IMAGE_SIZE = 28
IMAGE_SIZE_O4 = int(IMAGE_SIZE/4)

MAX_EPOCHS = 200000
BATCH_SIZE = 450

NOISE_DIM = 100

GENERATOR_LEARNING_RATE = 1e-4
DISCRIMINATOR_LEARNING_RATE = 1e-3

In [ ]:
# Define the generator model
def generator_model():
    # They happen in a linear order
    model = keras.Sequential()

    # Add a NN layer with 7*7*256 nodes and no bias
    model.add(keras.layers.Dense(IMAGE_SIZE_O4*IMAGE_SIZE_O4*256, use_bias=False, input_shape=(NOISE_DIM,)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.LeakyReLU())

    # Reshape back into a 2D image with 256 layers (and confirm it was reshaped correctly)
    model.add(keras.layers.Reshape((IMAGE_SIZE_O4, IMAGE_SIZE_O4, 256)))

    # Convolutional layer; ensures that the output will be the same size
    model.add(keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.LeakyReLU())

    # Another convolutional layer
    model.add(keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.LeakyReLU())
    
    # Convolution
    model.add(keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    
    # Print summary?
    print(model.summary())

    # Return
    return model

# Get the model
generator = generator_model()

In [ ]:
# Generates one image from the generator to use as an example
def generate_one_image():
  noise = tf.random.normal([1, NOISE_DIM])
  generated_images = generator(noise, training=False)
  return generated_images[0]
generate_one_image()

In [ ]:
def discriminator_model():
    # Define model
    model = tf.keras.Sequential()
    
    # Convolutional layer
    model.add(keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[IMAGE_SIZE, IMAGE_SIZE, 1]))
    model.add(keras.layers.LeakyReLU()) # ReLU should always come after a convolutional layer
    model.add(keras.layers.Dropout(0.3)) # Randomly sets 30% of nodes to 0 during training. Prevents overfitting.

    # Another convolution layer, same as above
    model.add(keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(keras.layers.LeakyReLU())
    model.add(keras.layers.Dropout(0.3))

    # Flatten to a 1D vector
    model.add(keras.layers.Flatten())

    # Final step is to convert into a single scalar representing rating
    model.add(keras.layers.Dense(1))

    print(model.summary())

    return model

# Get the model
discriminator = discriminator_model()

In [ ]:
def discriminator_loss(predicted_output, human_output):
    cross_entropy = tf.keras.losses.BinaryCrossentropy()
    return cross_entropy(predicted_output, human_output)

def generator_loss(fake_output):
    # Generator always wants a value of one (which indicates a high rating)
    cross_entropy = tf.keras.losses.BinaryCrossentropy()
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Declare optimizers
# Judge needs a much faster learning rate to make the most of human feedback
generator_optimizer = tf.keras.optimizers.Adam(GENERATOR_LEARNING_RATE)
judge_optimizer = tf.keras.optimizers.Adam(DISCRIMINATOR_LEARNING_RATE)